In [18]:
import numpy as np
import pandas as pd

In [19]:
__author__ = 'Chris Potts'


class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        The temperature parameter. Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon * self.prior)

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        spk = self.speaker().T
        return rownorm(spk * self.prior)


def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Examples"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['квадрат', 'треугольник', 'круг', 'ромб']
    states = ['r1', 'r2', 'r3', 'r4']
    lex = pd.DataFrame([
        [1.0, 0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0, 0.0],
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 0.0, 0.0, 1.0]], index=msgs, columns=states)

In [20]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,r4,costs
квадрат,1.00,0.0,0.0,1.0,0.0
треугольник,0.00,1.0,0.0,0.0,0.0
круг,0.00,0.0,1.0,0.0,0.0
ромб,0.00,0.0,0.0,1.0,0.0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Literal listener


,r1,r2,r3,r4
квадрат,0.5,0.0,0.0,0.5
треугольник,0.0,1.0,0.0,0.0
круг,0.0,0.0,1.0,0.0
ромб,0.0,0.0,0.0,1.0



Pragmatic speaker


,квадрат,треугольник,круг,ромб
r1,1.000000,0.0,0.0,0.000000
r2,0.000000,1.0,0.0,0.000000
r3,0.000000,0.0,1.0,0.000000
r4,0.333333,0.0,0.0,0.666667



Pragmatic listener


,r1,r2,r3,r4
квадрат,0.75,0.0,0.0,0.25
треугольник,0.00,1.0,0.0,0.00
круг,0.00,0.0,1.0,0.00
ромб,0.00,0.0,0.0,1.00


In [21]:
    print("="*70 + "\nEven priors, imbalanced message costs\n")
    cost_most = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[2.0, 0.0, 0.0, -6.0])

    display_reference_game(cost_most)

    print("\nLiteral listener")
    display(cost_most.literal_listener())

    print("\nPragmatic speaker")
    display(cost_most.speaker())

    print("\nPragmatic listener")
    display(cost_most.listener())

Even priors, imbalanced message costs



,r1,r2,r3,r4,costs
квадрат,1.00,0.0,0.0,1.0,2.0
треугольник,0.00,1.0,0.0,0.0,0.0
круг,0.00,0.0,1.0,0.0,0.0
ромб,0.00,0.0,0.0,1.0,-6.0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Literal listener


,r1,r2,r3,r4
квадрат,0.5,0.0,0.0,0.5
треугольник,0.0,1.0,0.0,0.0
круг,0.0,0.0,1.0,0.0
ромб,0.0,0.0,0.0,1.0



Pragmatic speaker


,квадрат,треугольник,круг,ромб
r1,1.00000,0.0,0.0,0.00000
r2,0.00000,1.0,0.0,0.00000
r3,0.00000,0.0,1.0,0.00000
r4,0.99933,0.0,0.0,0.00067



Pragmatic listener


,r1,r2,r3,r4
квадрат,0.500168,0.0,0.0,0.499832
треугольник,0.000000,1.0,0.0,0.000000
круг,0.000000,0.0,1.0,0.000000
ромб,0.000000,0.0,0.0,1.000000


In [28]:
    print("="*70 + "\nEven priors and all-0 message costs; alpha = 5\n")
    alpha_mod = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0], alpha=5.0)

    display_reference_game(alpha_mod)

    print("\nLiteral listener")
    display(alpha_mod.literal_listener())

    print("\nPragmatic speaker")
    display(alpha_mod.speaker())

    print("\nPragmatic listener")
    display(alpha_mod.listener())

Even priors and all-0 message costs; alpha = 5



,r1,r2,r3,r4,costs
квадрат,1.00,0.0,0.0,1.0,0.0
треугольник,0.00,1.0,0.0,0.0,0.0
круг,0.00,0.0,1.0,0.0,0.0
ромб,0.00,0.0,0.0,1.0,0.0
prior,0.25,0.25,0.25,0.25,
alpha,5.00,,,,



Literal listener


,r1,r2,r3,r4
квадрат,0.5,0.0,0.0,0.5
треугольник,0.0,1.0,0.0,0.0
круг,0.0,0.0,1.0,0.0
ромб,0.0,0.0,0.0,1.0



Pragmatic speaker


,квадрат,треугольник,круг,ромб
r1,1.000000,0.0,0.0,0.000000
r2,0.000000,1.0,0.0,0.000000
r3,0.000000,0.0,1.0,0.000000
r4,0.030303,0.0,0.0,0.969697



Pragmatic listener


,r1,r2,r3,r4
квадрат,0.970588,0.0,0.0,0.029412
треугольник,0.000000,1.0,0.0,0.000000
круг,0.000000,0.0,1.0,0.000000
ромб,0.000000,0.0,0.0,1.000000


In [23]:
    print("="*70 + "\nImbalanced priors, all-0 message costs\n")
    prior_mod = RSA(lexicon=lex, prior=[0.1, 0.1, 0.1, 0.7], costs=[0.0, 0.0, 0.0, 0.0])

    display_reference_game(prior_mod)

    print("\nLiteral listener")
    display(prior_mod.literal_listener())

    print("\nPragmatic speaker")
    display(prior_mod.speaker())

    print("\nPragmatic listener")
    display(prior_mod.listener())

Imbalanced priors, all-0 message costs



,r1,r2,r3,r4,costs
квадрат,1.0,0.0,0.0,1.0,0.0
треугольник,0.0,1.0,0.0,0.0,0.0
круг,0.0,0.0,1.0,0.0,0.0
ромб,0.0,0.0,0.0,1.0,0.0
prior,0.1,0.1,0.1,0.7,
alpha,1.0,,,,



Literal listener


,r1,r2,r3,r4
квадрат,0.125,0.0,0.0,0.875
треугольник,0.000,1.0,0.0,0.000
круг,0.000,0.0,1.0,0.000
ромб,0.000,0.0,0.0,1.000



Pragmatic speaker


,квадрат,треугольник,круг,ромб
r1,1.000000,0.0,0.0,0.000000
r2,0.000000,1.0,0.0,0.000000
r3,0.000000,0.0,1.0,0.000000
r4,0.466667,0.0,0.0,0.533333



Pragmatic listener


,r1,r2,r3,r4
квадрат,0.234375,0.0,0.0,0.765625
треугольник,0.000000,1.0,0.0,0.000000
круг,0.000000,0.0,1.0,0.000000
ромб,0.000000,0.0,0.0,1.000000
